<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/NLP/Kaggle_upvoted_dataset/Kaggle_NLP_upvoted_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/canggih/voted-kaggle-dataset/notebooks

In [2]:
import pandas as pd
import numpy as np  

In [3]:
data=pd.read_csv('https://github.com/duybluemind1988/Data-science/blob/master/NLP/Kaggle_upvoted_dataset/voted-kaggle-dataset.csv?raw=true')
print(data.shape)
data.head()

(2150, 15)


,Title,Subtitle,Owner,Votes,Versions,Tags,Data Type,Size,License,Views,Download,Kernels,Topics,URL,Description
0,Credit Card Fraud Detection,Anonymized credit card transactions labeled as...,Machine Learning Group - ULB,1241,"Version 2,2016-11-05|Version 1,2016-11-03",crime\nfinance,CSV,144 MB,ODbL,"442,136 views","53,128 downloads","1,782 kernels",26 topics,https://www.kaggle.com/mlg-ulb/creditcardfraud,The datasets contains transactions made by cre...
1,European Soccer Database,"25k+ matches, players & teams attributes for E...",Hugo Mathien,1046,"Version 10,2016-10-24|Version 9,2016-10-24|Ver...",association football\neurope,SQLite,299 MB,ODbL,"396,214 views","46,367 downloads","1,459 kernels",75 topics,https://www.kaggle.com/hugomathien/soccer,The ultimate Soccer database for data analysis...
2,TMDB 5000 Movie Dataset,"Metadata on ~5,000 movies from TMDb",The Movie Database (TMDb),1024,"Version 2,2017-09-28",film,CSV,44 MB,Other,"446,255 views","62,002 downloads","1,394 kernels",46 topics,https://www.kaggle.com/tmdb/tmdb-movie-metadata,Background\nWhat can we say about the success ...
3,Global Terrorism Database,"More than 170,000 terrorist attacks worldwide,...",START Consortium,789,"Version 2,2017-07-19|Version 1,2016-12-08",crime\nterrorism\ninternational relations,CSV,144 MB,Other,"187,877 views","26,309 downloads",608 kernels,11 topics,https://www.kaggle.com/START-UMD/gtd,"Context\nInformation on more than 170,000 Terr..."
4,Bitcoin Historical Data,Bitcoin data at 1-min intervals from select ex...,Zielak,618,"Version 11,2018-01-11|Version 10,2017-11-17|Ve...",history\nfinance,CSV,119 MB,CC4,"146,734 views","16,868 downloads",68 kernels,13 topics,https://www.kaggle.com/mczielinski/bitcoin-his...,Context\nBitcoin is the longest running and mo...


In [6]:
data.Description[0]

"The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.\nIt contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case o

# Ch06b - Topic Modeling with gensim.ipynb

## Basic Text Wrangling

In [10]:
papers=data.Description
print(len(papers))
papers

2150


0       The datasets contains transactions made by cre...
1       The ultimate Soccer database for data analysis...
2       Background\nWhat can we say about the success ...
3       Context\nInformation on more than 170,000 Terr...
4       Context\nBitcoin is the longest running and mo...
                              ...                        
2145    Context\nFortnite: Battle Royale has over 20 m...
2146    Context\nThis dataset provides the nationaliti...
2147    lem.json\nThis file contains lementized englis...
2148    Context\nThis data set contains weather data f...
2149    Context\nBirths in U.S during 1994 to 2003.\nC...
Name: Description, Length: 2150, dtype: object

In [12]:
papers=papers.astype(str)

In [13]:
%%time
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
  

stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
            
    return norm_papers
    
norm_papers = normalize_corpus(papers)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
CPU times: user 3.14 s, sys: 33.9 ms, total: 3.17 s
Wall time: 3.2 s


In [18]:
print(papers[0])
print(norm_papers[0])

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of 

## Text Representation with Feature Engineering

In [ ]:
import gensim

bigram = gensim.models.Phrases(norm_papers, min_count=20, threshold=20, delimiter=b'_') 
# higher threshold fewer phrases.
bigram_model = gensim.models.phrases.Phraser(bigram)

In [20]:
norm_corpus_bigrams = [bigram_model[doc] for doc in norm_papers]
print(norm_corpus_bigrams[0])

['datasets', 'contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'dataset', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'dataset', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'cannot', 'provide', 'original', 'feature', 'background', 'information', 'data', 'feature', 'v1', 'v2', 'v28', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'dataset', 'feature', 'amount', 'transaction', 'amount', 'feature', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring

In [21]:
# Create a dictionary representation of the documents.
# Dictionary encapsulates the mapping between normalized words and their integer ids.
dictionary = gensim.corpora.Dictionary(norm_corpus_bigrams)

In [22]:
print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, 'ac'), (1, 'account'), (2, 'accuracy'), (3, 'amount'), (4, 'analysed'), (5, 'andrea'), (6, 'area'), (7, 'artml'), (8, 'auprc'), (9, 'available'), (10, 'background'), (11, 'big'), (12, 'bontempi'), (13, 'brufence'), (14, 'bruxelles')]
Total Vocabulary Size: 32097


In [23]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.6)
print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, 'ac'), (1, 'account'), (2, 'accuracy'), (3, 'amount'), (4, 'area'), (5, 'available'), (6, 'background'), (7, 'big'), (8, 'cannot'), (9, 'card'), (10, 'case'), (11, 'class'), (12, 'classification'), (13, 'collected'), (14, 'component')]
Total Vocabulary Size: 1821


In [25]:
# Transforming corpus into bag of words vectors
bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigrams]
print(bow_corpus[0][:50])

[(0, 3), (1, 1), (2, 2), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 2), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 7), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)]


doc2bow: Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

If allow_update is set, then also update dictionary in the process: create ids for new words. At the same time, update document frequencies -- for each word appearing in this document, increase its document frequency (self.dfs) by one.

If allow_update is not set, this function is const, aka read-only.

In [26]:
print([(dictionary[idx] , freq) for idx, freq in bow_corpus[1][:50]])

[('card', 1), ('case', 1), ('class', 2), ('day', 1), ('european', 1), ('feature', 1), ('first', 1), ('ha', 3), ('http', 2), ('machine_learning', 1), ('original', 2), ('probability', 2), ('project', 1), ('time', 2), ('using', 2), ('value', 1), ('able', 2), ('access', 2), ('achieved', 1), ('across', 1), ('adding', 1), ('algorithm', 1), ('also', 3), ('analysis', 1), ('api', 1), ('appears', 1), ('ask', 1), ('asset', 1), ('attribute', 5), ('away', 1), ('base', 1), ('called', 1), ('changed', 1), ('classifier', 1), ('click', 2), ('co', 1), ('collection', 1), ('column', 1), ('com', 3), ('come', 1), ('commercial', 1), ('compare', 1), ('containing', 1), ('coordinate', 1), ('could', 1), ('country', 1), ('crawling', 2), ('cross', 1), ('database', 2), ('design', 1)]


In [27]:
print('Total number of papers:', len(bow_corpus))

Total number of papers: 2150


## Topic Models with Latent Semantic Indexing (LSI)

In [47]:
%%time
TOTAL_TOPICS = 20
lsi_bow = gensim.models.LsiModel(bow_corpus, id2word=dictionary, num_topics=TOTAL_TOPICS,
                                 onepass=True, chunksize=1740, power_iters=1000)

CPU times: user 2min, sys: 46.6 s, total: 2min 47s
Wall time: 1min 24s


In [48]:
for topic_id, topic in lsi_bow.print_topics(num_topics=20, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

Topic #1:
0.879*"university" + 0.124*"player" + 0.101*"number" + 0.091*"wa" + 0.090*"college" + 0.088*"year" + 0.088*"time" + 0.085*"file" + 0.079*"team" + 0.075*"csv" + 0.069*"one" + 0.066*"information" + 0.064*"name" + 0.063*"ha" + 0.060*"date" + 0.049*"value" + 0.046*"institute" + 0.045*"used" + 0.045*"new" + 0.045*"university_california"

Topic #2:
-0.452*"university" + 0.294*"player" + 0.207*"number" + 0.183*"team" + 0.176*"year" + 0.176*"time" + 0.172*"wa" + 0.168*"file" + 0.146*"csv" + 0.134*"one" + 0.130*"name" + 0.124*"date" + 0.122*"information" + 0.121*"ha" + 0.100*"goal" + 0.098*"value" + 0.088*"game" + 0.086*"code" + 0.085*"used" + 0.080*"use"

Topic #3:
-0.678*"player" + -0.394*"team" + -0.237*"goal" + -0.151*"zone" + 0.135*"file" + -0.134*"game" + -0.129*"allowed" + 0.110*"csv" + 0.102*"year" + 0.091*"one" + 0.087*"date" + 0.080*"information" + -0.076*"percentage" + 0.075*"name" + 0.072*"value" + -0.068*"individual" + -0.065*"taken" + -0.065*"scored" + -0.059*"relative" 

Let’s take a moment to understand these results. A brief recap on the LSI model— it is based on the principle that words that are used in the same contexts tend to have similar meanings. You can observe in this output that each topic is a combination of terms (which basically tend to convey an overall sense of the topic) and weights. Now the problem here is that we have both positive and negative weights. What does that mean?

Based on existing research and my interpretations, considering we are reducing the dimensionality here to a 10-dimensional space based on the number of topics, the sign on each term indicates a sense of direction or orientation in the vector space for a particular topic. The higher the weight, the more important the contribution. So similar correlated terms have the same sign or direction. Hence, it is perfectly possible for a topic to have two different sub-themes based on the sign or orientation of terms. Let’s separate these terms and try to interpret the topics again.

In [49]:
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    for term, wt in lsi_bow.show_topic(n, topn=20):
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term, round(wt, 3)))

    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()
    

Topic #1:
Direction 1: [('university', 0.879), ('player', 0.124), ('number', 0.101), ('wa', 0.091), ('college', 0.09), ('year', 0.088), ('time', 0.088), ('file', 0.085), ('team', 0.079), ('csv', 0.075), ('one', 0.069), ('information', 0.066), ('name', 0.064), ('ha', 0.063), ('date', 0.06), ('value', 0.049), ('institute', 0.046), ('used', 0.045), ('new', 0.045), ('university_california', 0.045)]
--------------------------------------------------
Direction 2: []
--------------------------------------------------

Topic #2:
Direction 1: [('player', 0.294), ('number', 0.207), ('team', 0.183), ('year', 0.176), ('time', 0.176), ('wa', 0.172), ('file', 0.168), ('csv', 0.146), ('one', 0.134), ('name', 0.13), ('date', 0.124), ('information', 0.122), ('ha', 0.121), ('goal', 0.1), ('value', 0.098), ('game', 0.088), ('code', 0.086), ('used', 0.085), ('use', 0.08)]
--------------------------------------------------
Direction 2: [('university', -0.452)]
----------------------------------------------

Does this make things better? Well, it’s definitely a lot better than the previous interpretation. Here we can see clear themes of modeling being applied in chips and electronic devices, classification and recognition models, neural models talking about the human brain components like cells, stimuli, neurons, cortical components, and even themes around reinforcement learning! We explore these in detail later in a more structured way. Let’s try to get the three major matrices (U, S, and VT) from our topic model, which uses SVD (based on the foundational concepts mentioned earlier)

In [50]:
term_topic = lsi_bow.projection.u
singular_values = lsi_bow.projection.s
topic_document = (gensim.matutils.corpus2dense(lsi_bow[bow_corpus], len(singular_values)).T / singular_values).T
term_topic.shape, singular_values.shape, topic_document.shape

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


((1821, 20), (20,), (20, 2150))

In [51]:
document_topics = pd.DataFrame(np.round(topic_document.T, 3), 
                               columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
print(document_topics.shape)
document_topics.head(15)

(2150, 20)


,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20
0,0.006,0.012,0.009,-0.002,0.000,0.002,0.000,0.014,-0.004,0.011,-0.012,0.000,-0.005,0.011,-0.002,-0.016,0.025,-0.005,0.017,0.000
1,0.020,0.043,-0.041,-0.002,0.014,0.009,0.004,0.027,0.005,0.003,-0.002,0.008,0.015,0.033,-0.006,0.003,0.064,-0.012,-0.017,-0.031
2,0.015,0.027,0.018,-0.007,0.007,0.012,-0.000,0.024,0.022,-0.015,-0.000,-0.016,0.015,0.012,-0.019,0.002,0.017,-0.000,-0.021,-0.005
3,0.032,0.035,0.023,-0.016,0.000,0.005,0.005,0.043,0.005,-0.031,0.047,-0.013,0.070,0.016,-0.036,0.032,-0.017,-0.012,-0.067,0.006
4,0.007,0.013,0.008,-0.004,0.019,-0.009,-0.008,-0.009,-0.009,0.004,0.010,0.006,-0.003,0.017,-0.003,-0.002,0.011,-0.008,-0.014,-0.008
5,0.016,0.032,0.020,-0.003,0.040,-0.011,-0.014,-0.010,-0.008,0.008,0.030,0.002,0.007,0.020,-0.005,-0.007,0.017,0.013,-0.013,-0.000
6,0.002,0.004,0.003,-0.001,0.001,0.001,0.001,0.007,0.001,0.002,0.000,-0.001,-0.002,0.003,0.001,0.001,0.004,-0.001,-0.001,0.000
7,0.002,0.004,0.003,-0.000,0.001,0.000,0.001,0.003,0.003,-0.001,0.005,0.001,0.004,0.000,0.004,-0.004,0.008,0.004,-0.002,-0.006
8,0.009,0.017,0.012,-0.009,0.016,-0.011,-0.008,-0.009,0.005,0.010,-0.011,-0.006,-0.009,0.012,0.008,-0.002,0.020,-0.008,-0.012,-0.004
9,0.009,0.017,0.007,0.006,0.004,0.002,0.000,0.014,-0.010,0.002,0.001,0.006,-0.002,-0.001,-0.018,0.004,0.008,0.002,0.002,-0.021


In [55]:
top_topics[0][1:]

'19'

In [57]:
document_numbers = [1, 5, 10]

for document_number in document_numbers:
    top_topics = list(document_topics.columns[np.argsort(-np.absolute(document_topics.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print('Topic model '+top_topics[0][1:]+':',lsi_bow.show_topic(int(top_topics[0][1:]), topn=20))
    print('Topic model '+top_topics[1][1:]+':',lsi_bow.show_topic(int(top_topics[1][1:]), topn=20))
    print()

Document #1:
Dominant Topics (top 3): ['T17', 'T2', 'T3']
Paper Summary:
The ultimate Soccer database for data analysis and machine learning
What you get:
+25,000 matches
+10,000 players
11 European Countries with their lead championship
Seasons 2008 to 2016
Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including the weekly updates
Team line up with squad formation (X, Y coordinates)
Betting odds from up to 10 providers
Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches
*16th Oct 201
Topic model 17: [('police', -0.2924384771617504), ('station', -0.24798426375102275), ('drug', 0.23643131690674565), ('plan', 0.21513466376974605), ('survey', 0.20661591026593037), ('rating', 0.19236671004567105), ('race', 0.1919343437447153), ('health', 0.18984901662405568), ('crime', -0.18405253536345698), ('fire', -0.14441848483960898), ('location', -0.1315384180137746), ('user', -0.1273879204036706), ('woman', -0.116649

## Topic Models with Latent Dirichlet Allocation (LDA)

In [58]:
%%time
lda_model = gensim.models.LdaModel(corpus=bow_corpus, id2word=dictionary, chunksize=1740, 
                                   alpha='auto', eta='auto', random_state=42,
                                   iterations=500, num_topics=TOTAL_TOPICS, 
                                   passes=20, eval_every=None)

CPU times: user 32.3 s, sys: 17.1 ms, total: 32.3 s
Wall time: 32.3 s


In [59]:
for topic_id, topic in lda_model.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

Topic #18:
0.061*"numeric" + 0.026*"temperature" + 0.024*"university_california" + 0.017*"traffic" + 0.016*"ic_uci" + 0.015*"data_set" + 0.015*"text" + 0.015*"edu_ml" + 0.015*"location" + 0.015*"http_archive" + 0.014*"wind" + 0.013*"fatality" + 0.013*"school" + 0.012*"uci_machine" + 0.012*"weather" + 0.011*"learning_repository" + 0.011*"incident" + 0.010*"acknowledgement" + 0.009*"inspiration" + 0.008*"predict"

Topic #15:
0.033*"year" + 0.025*"energy" + 0.021*"total" + 0.020*"animal" + 0.019*"india" + 0.017*"solar" + 0.016*"rate" + 0.014*"response" + 0.013*"outcome" + 0.013*"customer" + 0.012*"natural" + 0.012*"space" + 0.011*"earth" + 0.010*"range" + 0.009*"wa" + 0.008*"acknowledgement" + 0.008*"type" + 0.008*"death" + 0.008*"based" + 0.008*"inspiration"

Topic #20:
0.041*"crime" + 0.041*"time" + 0.038*"score" + 0.028*"attack" + 0.018*"number" + 0.018*"defense" + 0.014*"submission" + 0.014*"non" + 0.014*"csv" + 0.014*"request" + 0.014*"statistic" + 0.013*"image" + 0.013*"matrix" + 0.

In [60]:
topics_coherences = lda_model.top_topics(bow_corpus, topn=20)
avg_coherence_score = np.mean([item[1] for item in topics_coherences])
print('Avg. Coherence Score:', avg_coherence_score)

Avg. Coherence Score: -2.5180356822814263


In [61]:
topics_with_wts = [item[0] for item in topics_coherences]
print('LDA Topics with Weights')
print('='*50)
for idx, topic in enumerate(topics_with_wts):
    print('Topic #'+str(idx+1)+':')
    print([(term, round(wt, 3)) for wt, term in topic])
    print()

LDA Topics with Weights
Topic #1:
[('wa', 0.01), ('time', 0.009), ('information', 0.009), ('source', 0.008), ('many', 0.008), ('world', 0.008), ('acknowledgement', 0.008), ('inspiration', 0.007), ('available', 0.007), ('project', 0.007), ('ha', 0.007), ('database', 0.006), ('one', 0.006), ('date', 0.006), ('also', 0.006), ('use', 0.006), ('country', 0.006), ('report', 0.006), ('number', 0.005), ('people', 0.005)]

Topic #2:
[('file', 0.036), ('user', 0.026), ('movie', 0.025), ('csv', 0.024), ('tag', 0.015), ('rating', 0.013), ('id', 0.012), ('contains', 0.012), ('research', 0.011), ('use', 0.011), ('paper', 0.01), ('information', 0.01), ('ha', 0.009), ('one', 0.009), ('format', 0.009), ('value', 0.009), ('data_set', 0.008), ('used', 0.008), ('following', 0.007), ('system', 0.007)]

Topic #3:
[('image', 0.048), ('class', 0.016), ('label', 0.011), ('instance', 0.011), ('cell', 0.01), ('integer', 0.01), ('sample', 0.01), ('model', 0.01), ('number', 0.008), ('training', 0.008), ('using', 0

In [63]:
cv_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus, 
                                                      texts=norm_corpus_bigrams,
                                                      dictionary=dictionary, 
                                                      coherence='c_v')
avg_coherence_cv = cv_coherence_model_lda.get_coherence()

umass_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus, 
                                                         texts=norm_corpus_bigrams,
                                                         dictionary=dictionary, 
                                                         coherence='u_mass')
avg_coherence_umass = umass_coherence_model_lda.get_coherence()

perplexity = lda_model.log_perplexity(bow_corpus)

print('Avg. Coherence Score (Cv):', avg_coherence_cv)
print('Avg. Coherence Score (UMass):', avg_coherence_umass)
print('Model Perplexity:', perplexity)

Avg. Coherence Score (Cv): 0.42230731344440936
Avg. Coherence Score (UMass): -2.5180356822814263
Model Perplexity: -6.85161118426254


In [65]:
topics = [[(term, round(wt, 3)) 
               for term, wt in lda_model.show_topic(n, topn=20)] 
                   for n in range(0, lda_model.num_topics)]

In [66]:
for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print([term for term, wt in topic])
    print()

Topic #1:
['university', 'college', 'map', 'stop', 'institute', 'chicago', 'technology', 'file', 'washington', 'national', 'archive', 'new', 'record', 'population', 'district', 'project', 'center', 'research', 'state', 'analysis']

Topic #2:
['wa', 'time', 'information', 'source', 'many', 'world', 'acknowledgement', 'inspiration', 'available', 'project', 'ha', 'database', 'one', 'date', 'also', 'use', 'country', 'report', 'number', 'people']

Topic #3:
['image', 'class', 'label', 'instance', 'cell', 'integer', 'sample', 'model', 'number', 'training', 'using', 'file', 'classifier', 'example', 'different', 'wa', 'two', 'machine_learning', 'used', 'test']

Topic #4:
['word', 'review', 'language', 'corpus', 'name', 'file', 'http', 'license', 'text', 'code', 'english', 'copyright', 'version', 'com', 'list', 'use', 'org', 'metadata', 'unzip', 'frequency']

Topic #5:
['text', 'article', 'title', 'sentence', 'speech', 'file', 'post', 'line', 'wa', 'id', 'using', 'news', 'corpus', 'ha', 'used',

In [68]:
topics_df = pd.DataFrame([[term for term, wt in topic] 
                              for topic in topics], 
                         columns = ['Term'+str(i) for i in range(1, 21)],
                         index=['Topic '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20
Term1,university,wa,image,word,text,price,yet,file,tweet,attribute,car,column,acknowledgement,name,year,player,city,numeric,year,crime
Term2,college,time,class,review,article,csv,doe_description,user,wa,area,inspiration,activity,information,code,energy,team,time,temperature,country,time
Term3,map,information,label,language,title,de,collection,movie,com,variable,largest,sensor,health,value,total,game,information,university_california,survey,score
Term4,stop,source,instance,corpus,sentence,stock,text,csv,twitter,building,past_research,feature,wa,fire,animal,match,number,traffic,age,attack
Term5,institute,many,cell,name,speech,com,txt,tag,one,product,science_community,signal,state,county,india,season,location,ic_uci,number,number
Term6,chicago,world,integer,file,file,question,use,rating,time,per,acquired,right,ha,state,solar,goal,date,data_set,state,defense
Term7,technology,acknowledgement,sample,http,post,date,street,id,like,ha,front_world,using,inspiration,air,rate,wa,new_york,text,name,submission
Term8,file,inspiration,model,license,line,market,south,contains,many,type,question_want,left,government,row,response,com,property,edu_ml,variable,non
Term9,washington,available,number,text,wa,acknowledgement,american,research,inspiration,day,see_answered,driving,company,csv,outcome,table,acknowledgement,location,population,csv
Term10,national,project,training,code,id,http_www,csv,use,song,model,data_set,label,federal,day,customer,point,election,http_archive,month,request


In [69]:
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic])  
                              for topic in topics],
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, lda_model.num_topics+1)]
                         )
topics_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Terms per Topic
Topic1,"university, college, map, stop, institute, chicago, technology, file, washington, national, archive, new, record, population, district, project, center, research, state, analysis"
Topic2,"wa, time, information, source, many, world, acknowledgement, inspiration, available, project, ha, database, one, date, also, use, country, report, number, people"
Topic3,"image, class, label, instance, cell, integer, sample, model, number, training, using, file, classifier, example, different, wa, two, machine_learning, used, test"
Topic4,"word, review, language, corpus, name, file, http, license, text, code, english, copyright, version, com, list, use, org, metadata, unzip, frequency"
Topic5,"text, article, title, sentence, speech, file, post, line, wa, id, using, news, corpus, ha, used, inspiration, word, en, google, comment"
Topic6,"price, csv, de, stock, com, question, date, market, acknowledgement, http_www, day, en, inspiration, company, close, open, volume, la, index, per"
Topic7,"yet, doe_description, collection, text, txt, use, street, south, american, csv, library, number, city, name, digital, tool, http, license, open, new_york"
Topic8,"file, user, movie, csv, tag, rating, id, contains, research, use, paper, information, ha, one, format, value, data_set, used, following, system"
Topic9,"tweet, wa, com, twitter, one, time, like, many, inspiration, song, user, column, used, would, site, acknowledgement, people, could, find, nltk"
Topic10,"attribute, area, variable, building, product, per, ha, type, day, model, value, number, feature, cluster, using, use, database, one, food, point"


## Interpreting Topic Model Results

In [72]:
tm_results = lda_model[bow_corpus]

In [73]:
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] 
                     for topics in tm_results]
corpus_topics[:5]

[(9, 0.5704999),
 (15, 0.5066352),
 (1, 0.4261907),
 (1, 0.32978037),
 (5, 0.447578)]

In [74]:
corpus_topic_df = pd.DataFrame()
corpus_topic_df['Document'] = range(0, len(papers))
corpus_topic_df['Dominant Topic'] = [item[0]+1 for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
corpus_topic_df['Topic Desc'] = [topics_df.iloc[t[0]]['Terms per Topic'] for t in corpus_topics]
corpus_topic_df['Paper'] = papers

### Dominant Topics Distribution across Corpus

In [82]:
corpus_topic_df.head(2)

,Document,Dominant Topic,Contribution %,Topic Desc,Paper
0,0,10,57.05,"attribute, area, variable, building, product, per, ha, type, day, model, value, number, feature, cluster, using, use, database, one, food, point","The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284..."
1,1,16,50.66,"player, team, game, match, season, goal, wa, com, table, point, result, play, information, league, pre_trained, sport, use, played, model, win","The ultimate Soccer database for data analysis and machine learning\nWhat you get:\n+25,000 matches\n+10,000 players\n11 European Countries with their lead championship\nSeasons 2008 to 2016\nPlay..."


In [83]:
corpus_topic_df.columns

Index(['Document', 'Dominant Topic', 'Contribution %', 'Topic Desc', 'Paper'], dtype='object')

### Dominant Topics in Specific Research Papers

In [76]:
pd.set_option('display.max_colwidth', 200)
(corpus_topic_df[corpus_topic_df['Document']
                 .isin([681, 9, 392, 1622, 17, 
                        906, 996, 503, 13, 733])])

,Document,Dominant Topic,Contribution %,Topic Desc,Paper
9,9,10,43.52,"attribute, area, variable, building, product, per, ha, type, day, model, value, number, feature, cluster, using, use, database, one, food, point","This data set includes 721 Pokemon, including their number, name, first and second type, and basic stats: HP, Attack, Defense, Special Attack, Special Defense, and Speed. It has been of great use ..."
13,13,18,99.67,"numeric, temperature, university_california, traffic, ic_uci, data_set, text, edu_ml, location, http_archive, wind, fatality, school, uci_machine, weather, learning_repository, incident, acknowled...","A food products database\nOpen Food Facts is a free, open, collbarative database of food products from around the world, with ingredients, allergens, nutrition facts and all the tidbits of informa..."
17,17,4,53.98,"word, review, language, corpus, name, file, http, license, text, code, english, copyright, version, com, list, use, org, metadata, unzip, frequency","Context\nThis dataset consists of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and us..."
392,392,2,34.49,"wa, time, information, source, many, world, acknowledgement, inspiration, available, project, ha, database, one, date, also, use, country, report, number, people",Context\nBible (or Biblia in Greek) is a collection of sacred texts or scriptures that Jews and Christians consider to be a product of divine inspiration and a record of the relationship between G...
503,503,19,44.98,"year, country, survey, age, number, state, name, variable, population, month, source, information, education, student, gender, policy, acknowledgement, wa, data_set, rate","Description:\nHappyDB is a corpus of more than 100,000 happy moments crowd-sourced via Amazon’s Mechanical Turk.\nEach worker is given the following task: What made you happy today? Reflect on the..."
681,681,10,56.89,"attribute, area, variable, building, product, per, ha, type, day, model, value, number, feature, cluster, using, use, database, one, food, point","Build the proposed optimized pricing model: Determine the largest or key value driver from the data Build price segments using product characteristics, distribution channel, behavior and demograph..."
733,733,2,43.54,"wa, time, information, source, many, world, acknowledgement, inspiration, available, project, ha, database, one, date, also, use, country, report, number, people",Context\nThis dataset is a list of people who have been involved in an accident in the city of Barcelona (Spain) from year 2010 till 2016. This data is managed by the Police in the city of Barcelo...
906,906,13,33.67,"acknowledgement, information, health, wa, state, ha, inspiration, government, company, federal, service, published, patient, database, year, department, vehicle, risk, doe, gov","Context\nThe CalCOFI data set represents the longest (1949-present) and most complete (more than 50,000 sampling stations) time series of oceanographic and larval fish data in the world. It includ..."
996,996,9,64.77,"tweet, wa, com, twitter, one, time, like, many, inspiration, song, user, column, used, would, site, acknowledgement, people, could, find, nltk","Context\nAs a huge LOTR fan, I was excited to have acquired this character data from the Lord of the Rings Wiki. I scraped this data using F#; the repository can be found here: https://github.com/..."
1622,1622,9,44.45,"tweet, wa, com, twitter, one, time, like, many, inspiration, song, user, column, used, would, site, acknowledgement, people, could, find, nltk","Context\nI started to scrape TripAdvisor reviews for a personal project on Sentiment Analysis. I thought it could be good to share my data on Kaggle, since this can help other with similar ideas.\..."


### Relevant Research Papers per Topic based on Dominance

In [77]:
corpus_topic_df.groupby('Dominant Topic').apply(lambda topic_set: (topic_set.sort_values(by=['Contribution %'], 
                                                                                         ascending=False)
                                                                             .iloc[0]))

,Document,Dominant Topic,Contribution %,Topic Desc,Paper
Dominant Topic,,,,,
1,722,1,99.35,"university, college, map, stop, institute, chicago, technology, file, washington, national, archive, new, record, population, district, project, center, research, state, analysis","Context\nInformation reproduced from the National Archives:\n""The Vietnam Conflict Extract Data File of the Defense Casualty Analysis System (DCAS) Extract Files contains records of 58,220 U.S. mi..."
2,1930,2,98.50,"wa, time, information, source, many, world, acknowledgement, inspiration, available, project, ha, database, one, date, also, use, country, report, number, people",82.558 Human Instructions in Chinese Extracted from wikiHow\nStep-by-step instructions in Chinese extracted from wikiHow and decomposed into a formal graph representation in RDF.\nThis is one of m...
3,610,3,98.90,"image, class, label, instance, cell, integer, sample, model, number, training, using, file, classifier, example, different, wa, two, machine_learning, used, test","This dataset contains 16,000 images of four shapes; square, star, circle, and triangle. Each image is 200x200 pixels.\nThe data was collected using a Garmin Virb 1080p action camera. The shapes we..."
4,2081,4,97.81,"word, review, language, corpus, name, file, http, license, text, code, english, copyright, version, com, list, use, org, metadata, unzip, frequency",Context\nFastText word embeddings trained on English wikipedia\nFastText embeddings are enriched with sub-word information useful in dealing with misspelled and out-of-vocabulary words.\nContent\n...
5,1860,5,99.00,"text, article, title, sentence, speech, file, post, line, wa, id, using, news, corpus, ha, used, inspiration, word, en, google, comment",Context:\nYoutube has introduced automatic generation of subtitles based on speech recognition of uploaded video. This dataset provides collection of subtitles Robert Phoenix The 11th House upload...
6,812,6,99.59,"price, csv, de, stock, com, question, date, market, acknowledgement, http_www, day, en, inspiration, company, close, open, volume, la, index, per",«Datasets per la comparació de moviments i patrons entre els principals índexs borsatils espanyols i les crypto-monedes»\nContext\nEn aquest cas el context és detectar o preveure els diferents mov...
7,1197,7,99.17,"yet, doe_description, collection, text, txt, use, street, south, american, csv, library, number, city, name, digital, tool, http, license, open, new_york","Context\nOpenAddresses's goal is to connect the digital and physical worlds by sharing geographic coordinates, street names, house numbers and postal codes.\nContent\nThis dataset contains one dat..."
8,1963,8,98.54,"file, user, movie, csv, tag, rating, id, contains, research, use, paper, information, ha, one, format, value, data_set, used, following, system","Summary\nThis dataset (ml-20m) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 20000263 ratings and 465564 tag applications acros..."
9,2064,9,98.78,"tweet, wa, com, twitter, one, time, like, many, inspiration, song, user, column, used, would, site, acknowledgement, people, could, find, nltk","Context\nWalking around a Total Wine one day I wondered if there was any data I could find that would help me figure out what new rums to try, I later was able to find some information on rumratin..."


#Ch06c-Topic model with sk learn

In [89]:
data=pd.read_csv('https://github.com/duybluemind1988/Data-science/blob/master/NLP/Kaggle_upvoted_dataset/voted-kaggle-dataset.csv?raw=true')
print(data.shape)
data.head(1)

(2150, 15)


,Title,Subtitle,Owner,Votes,Versions,Tags,Data Type,Size,License,Views,Download,Kernels,Topics,URL,Description
0,Credit Card Fraud Detection,Anonymized credit card transactions labeled as fraudulent or genuine,Machine Learning Group - ULB,1241,"Version 2,2016-11-05|Version 1,2016-11-03",crime\nfinance,CSV,144 MB,ODbL,"442,136 views","53,128 downloads","1,782 kernels",26 topics,https://www.kaggle.com/mlg-ulb/creditcardfraud,"The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284..."


## Basic Text Wrangling

In [91]:
%%time
papers=papers.astype(str)
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
  

stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
            
    return norm_papers
    
norm_papers = normalize_corpus(papers)
print(len(norm_papers))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2150
CPU times: user 2.85 s, sys: 19.4 ms, total: 2.87 s
Wall time: 2.96 s


In [92]:
print(norm_papers[0])

['datasets', 'contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'dataset', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'dataset', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'cannot', 'provide', 'original', 'feature', 'background', 'information', 'data', 'feature', 'v1', 'v2', 'v28', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'dataset', 'feature', 'amount', 'transaction', 'amount', 'feature', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring

## Text Representation with Feature Engineering

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=20, max_df=0.6, ngram_range=(1,2),
                     token_pattern=None, tokenizer=lambda doc: doc,
                     preprocessor=lambda doc: doc)
cv_features = cv.fit_transform(norm_papers)
cv_features.shape

(2150, 2097)

In [94]:
vocabulary = np.array(cv.get_feature_names())
print('Total Vocabulary Size:', len(vocabulary))

Total Vocabulary Size: 2097


In [95]:
vocabulary

array(['1st', '2nd', 'ab', ..., 'zip code', 'zip file', 'zone'],
      dtype='<U26')

## Topic Models with Latent Semantic Indexing (LSI)

In [96]:
%%time
from sklearn.decomposition import TruncatedSVD

TOTAL_TOPICS = 20

lsi_model = TruncatedSVD(n_components=TOTAL_TOPICS, n_iter=500, random_state=42)
document_topics = lsi_model.fit_transform(cv_features)

CPU times: user 10 s, sys: 7.57 s, total: 17.6 s
Wall time: 9 s


In [97]:
document_topics.shape
# 2150 document

(2150, 20)

In [98]:
topic_terms = lsi_model.components_
topic_terms.shape
# 2097 words

(20, 2097)

In [99]:
topic_terms

array([[ 0.00080267,  0.00052692,  0.00037655, ...,  0.00028539,
         0.00027183,  0.00325737],
       [ 0.00885621,  0.00568069,  0.00364484, ...,  0.00214013,
         0.00257936,  0.0626348 ],
       [-0.00477828, -0.00230495, -0.00306242, ..., -0.00162598,
        -0.00236657,  0.1410079 ],
       ...,
       [-0.03070314, -0.02270857,  0.00195795, ...,  0.00182991,
         0.00139959, -0.04528696],
       [-0.01256348, -0.00855104, -0.00197959, ..., -0.00108223,
         0.00107812,  0.01667599],
       [-0.01868419, -0.01859952, -0.00289716, ..., -0.00260013,
        -0.0046619 ,  0.0088497 ]])

In [100]:
top_terms = 20
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms]
topic_keyterm_weights = np.array([topic_terms[row, columns] 
                             for row, columns in list(zip(np.arange(TOTAL_TOPICS), topic_key_term_idxs))])
topic_keyterms = vocabulary[topic_key_term_idxs]
topic_keyterms_weights = list(zip(topic_keyterms, topic_keyterm_weights))
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    terms, weights = topic_keyterms_weights[n]
    term_weights = sorted([(t, w) for t, w in zip(terms, weights)], 
                          key=lambda row: -abs(row[1]))
    for term, wt in term_weights:
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term, round(wt, 3)))

    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()

Topic #1:
Direction 1: [('university', 0.969), ('state', 0.175), ('college', 0.076), ('california', 0.049), ('university california', 0.041), ('institute', 0.039), ('new', 0.032), ('wa', 0.029), ('technology', 0.028), ('north', 0.027), ('year', 0.02), ('number', 0.02), ('file', 0.02), ('set', 0.02), ('san', 0.019), ('time', 0.018), ('st', 0.018), ('international', 0.018), ('csv', 0.017), ('player', 0.017)]
--------------------------------------------------
Direction 2: []
--------------------------------------------------

Topic #2:
Direction 1: [('player', 0.305), ('wa', 0.276), ('number', 0.21), ('team', 0.195), ('file', 0.189), ('time', 0.186), ('year', 0.174), ('csv', 0.17), ('one', 0.129), ('ha', 0.126), ('name', 0.125), ('information', 0.119), ('date', 0.115), ('http', 0.114), ('goal', 0.111), ('contains', 0.106), ('per', 0.1), ('value', 0.098), ('taken', 0.094)]
--------------------------------------------------
Direction 2: [('university', -0.121)]
-----------------------------

In [101]:
dt_df = pd.DataFrame(np.round(document_topics, 3), 
                     columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
dt_df.T
#2150 document

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149
T1,0.439,1.150,1.030,5.407,0.529,1.605,0.157,0.142,0.576,0.542,0.400,1.012,0.928,1.346,1.042,0.616,0.399,0.279,1.509,0.987,0.251,0.466,0.423,1.048,1.742,0.069,0.966,0.381,1.913,0.690,0.966,0.638,11.345,0.963,0.163,0.212,0.264,2.582,0.249,1.898,...,1.148,0.120,0.201,0.156,0.010,0.010,0.110,0.316,0.477,0.678,0.393,0.010,0.010,0.300,0.010,0.171,0.010,0.010,0.010,0.087,0.010,0.206,0.010,0.010,0.260,0.010,0.010,0.010,0.010,0.678,0.213,0.128,0.169,0.228,0.039,0.300,0.345,0.137,0.231,0.313
T2,3.561,11.376,7.611,11.192,4.449,10.668,1.292,1.104,4.714,4.545,1.973,5.762,5.910,8.704,2.449,3.017,3.270,2.161,8.132,7.626,1.769,3.959,3.672,11.006,12.280,0.496,8.143,3.122,17.566,5.982,7.000,5.311,2.433,3.395,1.386,1.674,2.162,4.401,2.170,5.936,...,2.439,0.816,1.022,1.204,0.077,0.077,0.930,2.082,3.908,4.460,2.820,0.077,0.077,1.201,0.077,0.761,0.077,0.077,0.077,0.810,0.077,1.524,0.077,0.077,1.625,0.077,0.077,0.077,0.077,4.460,1.625,0.939,1.449,1.825,0.323,2.731,2.829,1.221,1.950,2.301
T3,-2.190,5.920,-3.482,-5.392,-1.810,-4.841,-0.462,-0.678,-2.670,-1.829,-1.329,-2.146,-1.760,-8.425,-0.747,-0.887,-1.791,-1.486,-2.670,-3.975,-1.065,-1.029,-1.025,5.676,-4.683,-0.306,-5.266,-2.017,-6.781,-2.557,-4.512,-1.904,-0.969,-2.193,-0.813,-1.078,-0.729,-2.218,-1.300,-3.851,...,-0.676,-0.490,-0.643,-0.817,-0.053,-0.053,-0.505,-1.589,-2.013,-1.890,-0.958,-0.053,-0.053,-0.436,-0.053,-0.183,-0.053,-0.053,-0.053,-0.604,-0.053,-0.902,-0.053,-0.053,-0.851,-0.053,-0.053,-0.053,-0.053,-1.890,-0.880,-0.565,-0.740,-1.182,-0.234,-0.334,-1.312,-0.944,-0.465,-1.032
T4,-0.422,0.607,0.289,-1.468,-0.393,-0.388,-0.137,-0.100,-0.542,-0.077,-0.352,-0.137,-0.724,-2.483,0.124,-0.335,-0.394,-0.361,0.380,1.019,-0.212,-0.431,0.009,0.806,-1.078,-0.086,0.627,-0.407,-1.552,-0.417,-1.064,-0.090,-0.262,0.162,-0.199,0.063,-0.246,-0.277,-0.396,-0.125,...,-0.373,-0.122,-0.117,-0.186,-0.017,-0.017,-0.137,-0.344,-0.266,-0.372,-0.315,-0.017,-0.017,-0.213,-0.017,-0.014,-0.017,-0.017,-0.017,-0.137,-0.017,0.099,-0.017,-0.017,0.184,-0.017,-0.017,-0.017,-0.017,-0.372,-0.161,-0.096,0.113,0.130,-0.054,-0.320,-0.241,-0.178,-0.174,-0.332
T5,-0.224,-0.217,0.305,0.908,0.146,-0.107,-0.054,-0.129,0.607,-1.114,0.960,-1.768,-0.479,7.174,-2.690,-0.632,-0.500,-0.726,-3.546,-1.033,0.109,-0.710,1.033,0.114,-5.975,-0.239,1.954,0.815,-2.189,0.150,0.186,-0.855,-0.614,-1.846,0.195,-0.634,-0.011,-1.931,1.375,-0.194,...,-1.828,-0.644,-0.199,0.136,-0.038,-0.038,-0.276,0.173,-1.063,-0.959,-0.201,-0.038,-0.038,-0.222,-0.038,-0.268,-0.038,-0.038,-0.038,0.493,-0.038,-0.486,-0.038,-0.038,-1.104,-0.038,-0.038,-0.038,-0.038,-0.959,-0.503,-0.253,-0.115,0.005,0.132,-0.100,-0.799,0.480,-0.933,-1.552
T6,-0.853,-2.495,-1.340,-0.549,-2.863,-7.662,-0.389,-0.303,-2.387,-0.326,-0.269,-0.257,-2.359,-16.858,1.752,0.096,-0.251,-1.317,0.540,-3.537,-0.478,-2.767,0.974,-2.679,1.081,-0.046,-7.250,-1.684,6.712,-0.406,-7.606,-1.198,-0.866,-0.310,-0.399,-0.248,-1.102,-1.614,1.019,-2.381,...,0.728,0.146,-0.675,-0.852,0.004,0.004,0.344,-1.823,-2.471,-3.111,-0.057,0.004,0.004,0.955,0.004,-0.106,0.004,0.004,0.004,-0.561,0.004,-0.655,0.004,0.004,0.499,0.004,0.004,0.004,0.004,-3.111,-0.158,-0.283,-0.972,-1.186,-0.257,-0.488,-0.482,-0.524,0.859,1.609
T7,1.287,1.772,1.604,3.006,-2.224,-5.621,0.482,0.262,-1.642,0.829,-0.202,2.057,-2.406,76.337,-0.318,1.063,1.053,0.241,1.383,1.279,0.266,-0.658,0.505,1.254,1.660,0.173,-3.794,-2.289,-4.409,0.091,-7.400,2.531,0.964,0.694,-0.404,0.446,-0.027,1.843,0.039,9.500,...,0.334,0.080,0.224,0.520,0.008,0.008,-0.025,4.376,4.571,5.127,0.541,0.008,0.008,-0.134,0.008,0.261,0.008,0.008,0.008,-0.149,0.008,1.341,0.008,0.008,0.137,0.008,0.008,0.008,0.008,5.127,0.143,0.298,0.428,0.936,-0.020,0.405,0.454,0.418,-0.186,-

In [102]:
document_numbers = [1, 4, 10]

for document_number in document_numbers:
    top_topics = list(dt_df.columns[np.argsort(-np.absolute(dt_df.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print('Topic model '+top_topics[0][1:]+':',lsi_bow.show_topic(int(top_topics[0][1:]), topn=20))
    print('Topic model '+top_topics[1][1:]+':',lsi_bow.show_topic(int(top_topics[1][1:]), topn=20))
    print()

Document #1:
Dominant Topics (top 3): ['T2', 'T3', 'T19']
Paper Summary:
The ultimate Soccer database for data analysis and machine learning
What you get:
+25,000 matches
+10,000 players
11 European Countries with their lead championship
Seasons 2008 to 2016
Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including the weekly updates
Team line up with squad formation (X, Y coordinates)
Betting odds from up to 10 providers
Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches
*16th Oct 201
Topic model 2: [('player', -0.677578910958172), ('team', -0.3940464304148368), ('goal', -0.23656264928185738), ('zone', -0.15124711306019448), ('file', 0.1352717752315316), ('game', -0.13359756647119775), ('allowed', -0.12940669656171266), ('csv', 0.11046645363578558), ('year', 0.10156555786775204), ('one', 0.09087393933412283), ('date', 0.08701565699597512), ('information', 0.0804350089995299), ('percentage', -0.076279935

## Topic Models with Latent Dirichlet Allocation (LDA)

In [103]:
%%time
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components =TOTAL_TOPICS, max_iter=500, max_doc_update_iter=50,
                                      learning_method='online', batch_size=1740, learning_offset=50., 
                                      random_state=42, n_jobs=16)
document_topics = lda_model.fit_transform(cv_features)

CPU times: user 1min 1s, sys: 8.83 s, total: 1min 10s
Wall time: 7min 56s


In [104]:
topic_terms = lda_model.components_
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms] #20
topic_keyterms = vocabulary[topic_key_term_idxs]
topics = [', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame(topics,
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, TOTAL_TOPICS+1)])
topics_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,Terms per Topic
Topic1,"image, file, one, time, wa, contains, acknowledgement, inspiration, day, like, using, would, feature, used, set, see, available, column, label, different"
Topic2,"image, wa, time, text, survey, csv, acknowledgement, using, file, statistic, open, ha, numeric, set, value, price, http, learning, database, paper"
Topic3,"player, csv, team, game, file, wa, http, user, movie, time, match, com, number, ha, www, information, http www, set, id, rating"
Topic4,"context data, love, claim, basic, stats, inspiration, filed, al, east, acknowledgement, modification, john, lost, player, sometimes, file, contains, number, ha, year"
Topic5,"wa, state, year, information, acknowledgement, time, number, country, ha, available, inspiration, national, source, public, date, record, report, survey, includes, world"
Topic6,"text, word, language, file, corpus, id, http, use, wa, title, contains, article, license, question, english, page, name, org, author, collection"
Topic7,"set, data set, value, column, activity, wa, mean, energy, model, sample, feature, using, use, used, research, sensor, attribute, system, temperature, specie"
Topic8,"com, http, tweet, github, github com, twitter, http github, vector, http www, www, word, nltk, model, kaggle, acknowledgement, found, zip, code, tree, used"
Topic9,"year, age, type, month, animal, rate, sex, male, female, outcome, bank, birth, name, credit, number, payment, set, amount, usd, data set"
Topic10,"model, trained, de, pre, pre trained, trained model, feature, network, ha, depth, layer, deep, architecture, time, image, learned, large, en, model pre, learned feature"


In [105]:
pd.options.display.float_format = '{:,.3f}'.format
dt_df = pd.DataFrame(document_topics, 
                     columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
dt_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149
T1,0.416,0.214,0.413,0.067,0.226,0.109,0.243,0.001,0.429,0.000,0.001,0.506,0.000,0.081,0.001,0.202,0.240,0.001,0.000,0.646,0.001,0.091,0.394,0.045,0.000,0.003,0.110,0.401,0.334,0.097,0.000,0.684,0.000,0.074,0.368,0.452,0.001,0.080,0.245,0.010,...,0.001,0.002,0.001,0.978,0.007,0.007,0.966,0.001,0.000,0.000,0.001,0.007,0.007,0.788,0.007,0.908,0.007,0.007,0.007,0.003,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.436,0.001,0.874,0.905,0.008,0.001,0.764,0.002,0.720,0.001
T2,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.001,0.000,...,0.001,0.002,0.001,0.001,0.007,0.007,0.002,0.001,0.000,0.000,0.001,0.007,0.007,0.002,0.007,0.002,0.007,0.007,0.007,0.003,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.001,0.001,0.001,0.001,0.008,0.001,0.001,0.002,0.001,0.001
T3,0.000,0.775,0.291,0.070,0.267,0.361,0.002,0.001,0.271,0.715,0.235,0.035,0.114,0.000,0.033,0.001,0.000,0.333,0.060,0.000,0.687,0.443,0.144,0.746,0.199,0.003,0.678,0.166,0.151,0.335,0.483,0.024,0.000,0.000,0.001,0.001,0.186,0.000,0.082,0.000,...,0.001,0.002,0.001,0.001,0.007,0.007,0.002,0.001,0.000,0.000,0.154,0.007,0.007,0.002,0.007,0.002,0.007,0.007,0.007,0.571,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.001,0.001,0.001,0.001,0.008,0.851,0.001,0.002,0.001,0.001
T4,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.001,0.000,0.019,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.001,0.000,0.009,0.001,0.000,0.017,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.000,0.006,0.000,0.000,0.001,0.001,0.001,0.000,0.001,0.000,...,0.001,0.002,0.001,0.001,0.007,0.007,0.002,0.001,0.000,0.000,0.001,0.007,0.007,0.002,0.007,0.033,0.007,0.007,0.007,0.003,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.001,0.001,0.001,0.001,0.008,0.001,0.001,0.002,0.030,0.001
T5,0.000,0.000,0.268,0.764,0.210,0.321,0.002,0.934,0.000,0.000,0.515,0.106,0.345,0.000,0.501,0.304,0.000,0.001,0.772,0.000,0.302,0.218,0.230,0.097,0.524,0.213,0.000,0.050,0.000,0.000,0.334,0.112,0.646,0.587,0.001,0.085,0.377,0.095,0.113,0.030,...,0.505,0.752,0.509,0.001,0.007,0.007,0.002,0.001,0.000,0.000,0.173,0.007,0.007,0.179,0.007,0.002,0.007,0.007,0.007,0.003,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.372,0.001,0.001,0.001,0.008,0.001,0.001,0.002,0.001,0.383
T6,0.072,0.000,0.000,0.080,0.000,0.070,0.002,0.001,0.152,0.076,0.001,0.119,0.000,0.275,0.001,0.001,0.000,0.309,0.000,0.242,0.001,0.077,0.000,0.050,0.000,0.003,0.187,0.001,0.000,0.562,0.000,0.030,0.000,0.066,0.001,0.295,0.306,0.179,0.161,0.000,...,0.001,0.002,0.001,0.001,0.007,0.007,0.002,0.989,0.000,0.000,0.415,0.007,0.007,0.002,0.007,0.002,0.007,0.007,0.007,0.003,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.001,0.001,0.001,0.001,0.008,0.001,0.001,0.966,0.001,0.001
T7,0.324,0.000,0.000,0.000,0.000,0.022,0.465,0.001,0.000,0.000,0.001,0.000,0.468,0.000,0.001,0.132,0.214,0.001,0.162,0.000,0.001,0.000,0.169,0.000,0.031,0.003,0.000,0.001,0.000,0.000,0.000,0.108,0.000,0.000,0.001,0.034,0.001,0.523,0.001,0.316,...,0.134,0.002,0.001,0.001,0.007,0.007,0.002,0.001,0.000,0.000,0.001,0.007,0.007,0.002,0.007,0.002,0.007,0.007,0.007,0.003,0.007,0.001,0.007,0.007,0.001,0.007,0.007,0.007,0.007,0.000,0.001,0.976,0.001,0.001,0.008,0.001,0.001,0.002,0.001,0.001
T8,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.001,0.000,0.000,0.001,0.054,0.000,0.000,0.001,0.001,0.000,0.001,0.000,0.037,0.001,0.163,0.000,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.089,0.000,0.053,0.000,...,0.001,0.002,0.001,0.001,0.007

In [106]:
pd.options.display.float_format = '{:,.5f}'.format
pd.set_option('display.max_colwidth', 200)

max_contrib_topics = dt_df.max(axis=0)
dominant_topics = max_contrib_topics.index
contrib_perc = max_contrib_topics.values
document_numbers = [dt_df[dt_df[t] == max_contrib_topics.loc[t]].index[0]
                       for t in dominant_topics]
documents = [papers[i] for i in document_numbers]

results_df = pd.DataFrame({'Dominant Topic': dominant_topics, 'Contribution %': contrib_perc,
                          'Paper Num': document_numbers, 'Topic': topics_df['Terms per Topic'], 
                          'Paper Name': documents})
results_df

,Dominant Topic,Contribution %,Paper Num,Topic,Paper Name
Topic1,T1,0.99435,1977,"image, file, one, time, wa, contains, acknowledgement, inspiration, day, like, using, would, feature, used, set, see, available, column, label, different","Context\nWhile studying neural networks in machine learning, I found an ingenious 2-D scatter pattern at the cn231 course by Andrej Karpathy. Decision boundaries for the three classes of points ca..."
Topic2,T2,0.05000,589,"image, wa, time, text, survey, csv, acknowledgement, using, file, statistic, open, ha, numeric, set, value, price, http, learning, database, paper",nan
Topic3,T3,0.99554,1881,"player, csv, team, game, file, wa, http, user, movie, time, match, com, number, ha, www, information, http www, set, id, rating","Context\nThis dataset was built as a supplementary to ""[European Soccer Database][1]"". It includes data dictionary, extraction of detailed match information previously contains in XML columns.\nCo..."
Topic4,T4,0.91364,2037,"context data, love, claim, basic, stats, inspiration, filed, al, east, acknowledgement, modification, john, lost, player, sometimes, file, contains, number, ha, year",Context\nData of hitters in MLB's AL East\nContent\nBasic fundamental data on AL East hitters sorted descending by Plate Appearances\nAcknowledgements\nInspiration\nI love baseball and stats.
Topic5,T5,0.99195,1811,"wa, state, year, information, acknowledgement, time, number, country, ha, available, inspiration, national, source, public, date, record, report, survey, includes, world","Context:\nMapping the Klan is a rough timeline of the rise of the second Ku Klux Klan between 1915 and 1940. Each red dot shows a local unit or ""Klavern."" The official numbers for each Klavern ind..."
Topic6,T6,0.99379,1912,"text, word, language, file, corpus, id, http, use, wa, title, contains, article, license, question, english, page, name, org, author, collection","Context:\nSome words, like “the” or “and” in English, are used a lot in speech and writing. For most Natural Language Processing applications, you will want to remove these very frequent words. Th..."
Topic7,T7,0.99525,1986,"set, data set, value, column, activity, wa, mean, energy, model, sample, feature, using, use, used, research, sensor, attribute, system, temperature, specie",This research study was conducted to analyze the (potential) relationship between hardware and data set sizes. 100 data scientists from France between Jan-2016 and Aug-2016 were interviewed in ord...
Topic8,T8,0.98827,1870,"com, http, tweet, github, github com, twitter, http github, vector, http www, www, word, nltk, model, kaggle, acknowledgement, found, zip, code, tree, used",Context\nThe maxent_ne_chunker contains two pre-trained English named entity chunkers trained on an ACE corpus (perhaps ACE ACE 2004 Multilingual Training Corpus?)\nIt will load an nltk.chunk.name...
Topic9,T9,0.99472,1841,"year, age, type, month, animal, rate, sex, male, female, outcome, bank, birth, name, credit, number, payment, set, amount, usd, data set","Context\nThe Austin Animal Center is the largest no-kill animal shelter in the United States that provides care and shelter to over 18,000 animals each year and is involved in a range of county, c..."
Topic10,T10,0.99625,1355,"model, trained, de, pre, pre trained, trained model, feature, network, ha, depth, layer, deep, architecture, time, image, learned, large, en, model pre, learned feature",InceptionV3\nRethinking the Inception Architecture for Computer Vision\nConvolutional networks are at the core of most state-of-the-art computer vision solutions for a wide variety of tasks. Since...


## Topic Models with Non-Negative Matrix Factorization (NMF)

In [147]:
%%time
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=TOTAL_TOPICS, solver='cd', max_iter=500,
                random_state=42, alpha=.1, l1_ratio=.85)
document_topics = nmf_model.fit_transform(cv_features)

CPU times: user 4.73 s, sys: 226 ms, total: 4.96 s
Wall time: 4.72 s


In [148]:
topic_terms = nmf_model.components_
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms]
topic_keyterms = vocabulary[topic_key_term_idxs]
topics = [', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame(topics,
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, TOTAL_TOPICS+1)])
topics_df

,Terms per Topic
Topic1,"university, state, college, california, university california, institute, technology, north, new, san, st, south, international, washington, john, school, west, tech, science, chicago"
Topic2,"http, wa, information, com, set, www, acknowledgement, column, available, source, use, http www, also, ha, inspiration, one, contains, country, data set, match"
Topic3,"player, team, wa, goal, attempt, taken, weighted, zone, game, allowed, minute, per, average, individual, percentage, scored, relative, point, expected, hit"
Topic4,"integer, interested, enjoy, much, movie, categorical, people, always, music, interest, preference, lot, item, money, point, thing, life, often, make, time"
Topic5,"date, element, one, registration, number, zero, time, tag, end, start, application, file, containing, position, code, mark, version, field, section, status"
Topic6,"year, total, given, population, energy, percent, million, state, consumption, net, average, usd, billion, old, year year, rate, age, expenditure, period, people"
Topic7,"numeric, text, reading, real, reference, open, food, product, student, sensor, database, fact, binary, left, label, school, grade, right, turn, class"
Topic8,"csv, file, csv file, higher, india, contains, education, row, statistic, ha, name, match, source, one, street, score, like, license, following, included"
Topic9,"model, trained, pre, pre trained, feature, trained model, network, layer, ha, time, learned, model pre, transferable, learned feature, deep, architecture, depth, imagenet, large, using"
Topic10,"value, mean, name, hour, event, monitor, air, site, sample, parameter, number, code, measured, monitoring, summary, day, max, wa, standard, time"


In [109]:
pd.options.display.float_format = '{:,.3f}'.format
dt_df = pd.DataFrame(document_topics, 
                     columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
dt_df.head(10)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20
0,0.000,0.195,0.000,0.000,0.019,0.000,0.000,0.000,0.208,0.000,0.021,0.065,0.000,0.072,0.000,0.000,0.063,0.255,0.000,0.026
1,0.000,0.665,0.709,0.047,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.032,0.025,0.000,0.000
2,0.001,0.476,0.013,0.058,0.089,0.023,0.000,0.000,0.033,0.096,0.010,0.000,0.374,0.021,0.003,0.002,0.042,0.000,0.000,0.001
3,0.184,0.636,0.000,0.000,0.218,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.402,0.000,0.000,0.548,0.210,0.000,0.047,0.178
4,0.000,0.214,0.023,0.000,0.031,0.000,0.002,0.358,0.011,0.036,0.040,0.000,0.003,0.000,0.000,0.016,0.128,0.013,0.006,0.000
5,0.010,0.357,0.067,0.038,0.031,0.035,0.000,0.801,0.000,0.000,0.024,0.000,0.171,0.078,0.000,0.070,0.143,0.187,0.000,0.375
6,0.000,0.088,0.008,0.000,0.002,0.000,0.000,0.000,0.007,0.027,0.000,0.000,0.016,0.040,0.026,0.000,0.000,0.005,0.000,0.000
7,0.000,0.101,0.000,0.000,0.013,0.008,0.000,0.005,0.000,0.000,0.000,0.000,0.010,0.012,0.007,0.000,0.000,0.011,0.000,0.033
8,0.000,0.193,0.000,0.000,0.110,0.059,0.006,0.279,0.013,0.061,0.000,0.000,0.088,0.083,0.001,0.000,0.000,0.108,0.000,0.000
9,0.000,0.217,0.001,0.017,0.000,0.011,0.000,0.007,0.029,0.122,0.110,0.000,0.032,0.109,0.000,0.004,0.128,0.000,0.025,0.167


In [110]:
pd.options.display.float_format = '{:,.5f}'.format
pd.set_option('display.max_colwidth', 200)

max_score_topics = dt_df.max(axis=0)
dominant_topics = max_score_topics.index
term_score = max_score_topics.values
document_numbers = [dt_df[dt_df[t] == max_score_topics.loc[t]].index[0]
                       for t in dominant_topics]
documents = [papers[i] for i in document_numbers]

results_df = pd.DataFrame({'Dominant Topic': dominant_topics, 'Max Score': term_score,
                          'Paper Num': document_numbers, 'Topic': topics_df['Terms per Topic'], 
                          'Paper Name': documents})
results_df

,Dominant Topic,Max Score,Paper Num,Topic,Paper Name
Topic1,T1,16.32296,2071,"university, state, college, california, university california, institute, technology, north, new, san, st, south, international, washington, john, school, west, tech, science, chicago",Context\nData was grabbed from US-News: https://www.usnews.com\nThe following data points are included in this data set:\nRanking\nAcceptance-Rate\nAct-Avg\nSat-Avg\nPhoto\nCost after Financial Ai...
Topic2,T2,1.63533,585,"http, wa, information, com, set, www, acknowledgement, column, available, source, use, http www, also, ha, inspiration, one, contains, country, data set, match",About the Missing Migrants Data\nThis data is sourced from the International Organization for Migration. The data is part of a specific project called the Missing Migrants Project which tracks dea...
Topic3,T3,14.46719,627,"player, team, wa, goal, attempt, taken, weighted, zone, game, allowed, minute, per, average, individual, percentage, scored, relative, point, expected, hit",Context & Content\nThis dataset features the salaries of 874 nhl players for the 2016/2017 season. I have randomly split the players into a training (612 players) and test (262 players) population...
Topic4,T4,13.65851,41,"integer, interested, enjoy, much, movie, categorical, people, always, music, interest, preference, lot, item, money, point, thing, life, often, make, time","Introduction\nIn 2013, students of the Statistics class at FSEV UK were asked to invite their friends to participate in this survey.\nThe data file (responses.csv) consists of 1010 rows and 150 co..."
Topic5,T5,12.93043,1469,"date, element, one, registration, number, zero, time, tag, end, start, application, file, containing, position, code, mark, version, field, section, status","Context:\nThis dataset contains pending and registered trademark text data (no drawings/images) to include word mark, serial number, registration number, filing date, registration date, goods and ..."
Topic6,T6,7.93463,238,"year, total, given, population, energy, percent, million, state, consumption, net, average, usd, billion, old, year year, rate, age, expenditure, period, people",The purpose of this data set is to allow exploration between various types of data that is commonly collected by the US government across the states and the USA as a whole. The data set consists o...
Topic7,T7,8.89455,13,"numeric, text, reading, real, reference, open, food, product, student, sensor, database, fact, binary, left, label, school, grade, right, turn, class","A food products database\nOpen Food Facts is a free, open, collbarative database of food products from around the world, with ingredients, allergens, nutrition facts and all the tidbits of informa..."
Topic8,T8,6.12363,507,"csv, file, csv file, higher, india, contains, education, row, statistic, ha, name, match, source, one, street, score, like, license, following, included","Connect/Follow me on LinkedIn for more updates on interesting dataset like this. Thanks.\nContext\nMinistry of Human Resource Development (MHRD), Govt of India has initiated an All India Survey on..."
Topic9,T9,2.10993,1357,"model, trained, pre, pre trained, feature, trained model, network, layer, ha, time, learned, model pre, transferable, learned feature, deep, architecture, depth, imagenet, large, using","DenseNet-201\nDensely Connected Convolutional Networks\nRecent work has shown that convolutional networks can be substantially deeper, more accurate, and efficient to train if they contain shorter..."
Topic10,T10,7.23471,416,"value, mean, name, hour, event, monitor, air, site, sample, parameter, number, code, measured, monitoring, summary, day, max, wa, standard, time",Context:\nThe Environmental Protection Agency (EPA) creates air quality trends using measurements from monitors located across the country. All of this data comes from EPA’s Air Quality System (AQ...


## Persisting Model and Transformers

This is just for visualizing the topics in the other notebook (since PyLDAViz expands the notebook size)

In [112]:
import dill

with open('nmf_model.pkl', 'wb') as f:
    dill.dump(nmf_model, f)
with open('cv_features.pkl', 'wb') as f:
    dill.dump(cv_features, f)
with open('cv.pkl', 'wb') as f:
    dill.dump(cv, f)

# Ch06d - Visualizing Topic Models with pyLDAvis.ipynb

In [114]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 17.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=b920c9aa31ce1fd04349b206e8fd3a5714a8245fdcca3f603a8a2ff700953c69
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=45858e232b86e35ade290994127177ddb0b1f039494b9eb416abaf7cd8bf9561
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [115]:
import pyLDAvis
import pyLDAvis.sklearn
import dill
import warnings

warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

In [116]:
with open('nmf_model.pkl', 'rb') as f:
    nmf_model = dill.load(f)
with open('cv_features.pkl', 'rb') as f:
    cv_features = dill.load(f)
with open('cv.pkl', 'rb') as f:
    cv = dill.load(f)

In [117]:
import pandas as pd
import numpy as np

top_terms = 20
TOTAL_TOPICS = 20
vocabulary = np.array(cv.get_feature_names())
topic_terms = nmf_model.components_
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms]
topic_keyterms = vocabulary[topic_key_term_idxs]
topics = [', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame(topics,
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, TOTAL_TOPICS+1)])
topics_df

,Terms per Topic
Topic1,"university, state, college, california, university california, institute, technology, north, new, san, st, south, international, washington, john, school, west, tech, science, chicago"
Topic2,"http, wa, information, com, set, www, acknowledgement, column, available, source, use, http www, also, ha, inspiration, one, contains, country, data set, match"
Topic3,"player, team, wa, goal, attempt, taken, weighted, zone, game, allowed, minute, per, average, individual, percentage, scored, relative, point, expected, hit"
Topic4,"integer, interested, enjoy, much, movie, categorical, people, always, music, interest, preference, lot, item, money, point, thing, life, often, make, time"
Topic5,"date, element, one, registration, number, zero, time, tag, end, start, application, file, containing, position, code, mark, version, field, section, status"
Topic6,"year, total, given, population, energy, percent, million, state, consumption, net, average, usd, billion, old, year year, rate, age, expenditure, period, people"
Topic7,"numeric, text, reading, real, reference, open, food, product, student, sensor, database, fact, binary, left, label, school, grade, right, turn, class"
Topic8,"csv, file, csv file, higher, india, contains, education, row, statistic, ha, name, match, source, one, street, score, like, license, following, included"
Topic9,"model, trained, pre, pre trained, feature, trained model, network, layer, ha, time, learned, model pre, transferable, learned feature, deep, architecture, depth, imagenet, large, using"
Topic10,"value, mean, name, hour, event, monitor, air, site, sample, parameter, number, code, measured, monitoring, summary, day, max, wa, standard, time"


In [123]:
cv_features.shape

(2150, 2097)

In [118]:
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='mmds')

ValidationError: ignored

In [119]:
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='tsne')

ValidationError: ignored

In [ ]:
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='pcoa')

# Visualize from KAGGLE

https://www.kaggle.com/canggih/topic-modeling

Topic Modeling
Topic modeling is a statistical model to discover the abstract "topics" that occur in a collection of documents.
It is commonly used in text document. But nowadays, in social media analysis, topic modeling is an emerging research area.
One of the most popular algorithms used is Latent Dirichlet Allocation which was proposed by
David Blei et al in 2003.
Here, I want to perform topic modeling for the upvoted kaggle dataset.

Some notes on topic modeling:

To determine the number topics, it is common to use elbow method with perplexity score as its cost function.
To evaluate the models, we can calculate topic coherence.
Finally, to interpret the topics, as studied in social science research, there is triangulation metho

In [128]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=dc2c48581b2504d3a565ad5b592e854c141b8f366154c1655b1f8b433c506bb6
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [129]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim

In [130]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()

In [133]:
df=pd.read_csv('https://github.com/duybluemind1988/Data-science/blob/master/NLP/Kaggle_upvoted_dataset/voted-kaggle-dataset.csv?raw=true')
print(data.shape)
# sample data
print(df['Description'].head(2))

(2150, 15)
0    The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.\nIt contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes va

## Perform Tokenization, Words removal, and Lemmatization

In [135]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in df['Description'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)

# sample data
print(texts[0])

['datasets', 'contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'dataset', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'dataset', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'provide', 'original', 'feature', 'background', 'information', 'data', 'feature', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'dataset', 'feature', 'amount', 'transaction', 'amount', 'feature', 'can', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring', 'accuracy', 'using'

In [136]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [140]:
print(corpus[0][:20])

[(0, 3), (1, 1), (2, 2), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [142]:
len(corpus)
# 2150 description được biểu diễn bằng word_index and frequency

2150

## Generate LDA model

In [143]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=5))

[([(0.0329417, 'data'),
   (0.0123863565, 'content'),
   (0.0108074015, 'context'),
   (0.010650745, 'dataset'),
   (0.009167448, 'acknowledgement')],
  -0.32887626989415525),
 ([(0.024178976, 'price'),
   (0.022297142, 'csv'),
   (0.011118084, 'data'),
   (0.0094378255, 'dataset'),
   (0.0083645955, 'context')],
  -0.4169131867653458),
 ([(0.020295177, 'suny'),
   (0.008973113, 'southeastern'),
   (0.008972908, 'denver'),
   (0.006755966, 'xavier'),
   (0.0059344852, 'binghamton')],
  -0.7284820897876937),
 ([(0.020129709, 'dataset'),
   (0.013130812, 'data'),
   (0.011543181, 'can'),
   (0.011151869, 'file'),
   (0.008492353, 'contains')],
  -0.7868496335888502),
 ([(0.033303306, 'data'),
   (0.011286829, 'time'),
   (0.009875901, 'dataset'),
   (0.00782922, 'name'),
   (0.0076212115, 'city')],
  -1.3336312210573988),
 ([(0.016599022, 'de'),
   (0.015345448, 'song'),
   (0.012352671, 'data'),
   (0.009723297, 'row'),
   (0.008581463, 'csv')],
  -1.6178652324756506),
 ([(0.028868373, 

## Visualize the topic model

In [144]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=             x        y  topics  cluster     Freq
topic                                            
13    -0.18321 0.02016   1       1       26.05239
14    -0.16052 0.04208   2       1       16.50779
7     -0.15771 0.10911   3       1       15.52877
3     -0.16985 0.03964   4       1       12.97389
8     -0.02672 -0.01714  5       1       4.05159 
11    -0.02483 0.00629   6       1       3.87243 
1     0.17341  -0.09436  7       1       3.37515 
5     -0.10172 -0.00504  8       1       3.28446 
2     0.02169  -0.09052  9       1       3.18060 
12    0.00819  -0.08202  10      1       2.85960 
6     0.02572  -0.06414  11      1       2.25846 
4     0.03482  -0.04311  12      1       2.11446 
10    0.25268  0.32064   13      1       2.08576 
0     0.09622  -0.04066  14      1       1.36834 
9     0.21185  -0.10094  15      1       0.48631 , topic_info=             Term        Freq       Total Category  logprob  loglift
768   university  1,459.00000 1,459.00000  Default 30.00000 30.00000
38    dataset     4,267.00000 4,267.00000  Default 29.00000 29.00000
747   state       846.00000   846.00000    Default 28.00000 28.00000
1033  description 662.00000   662.00000    Default 27.00000 27.00000
801   csv         1,178.00000 1,178.00000  Default 26.00000 26.00000
...   ...                 ...         ...      ...      ...      ...
3184  facebook    4.10369     30.40747     Topic15 -5.91400 3.32330 
2186  en          4.87243     88.39231     Topic15 -5.74220 2.42790 
3067  flavor      2.69066     12.94818     Topic15 -6.33610 3.75490 
139   champion    2.44917     10.24934     Topic15 -6.43010 3.89460 
2342  google      2.68199     136.59718    Topic15 -6.33930 1.39560 

[1006 rows x 6 columns], token_table=       Topic    Freq       Term
term                           
12627  10    0.94307  abandon  
3355   6     0.41908  abnormal 
3355   14    0.53338  abnormal 
8725   13    0.25128  academy  
8725   14    0.69101  academy  
...    ..        ...      ...  
3161   4     0.26577  york     
3161   13    0.01969  york     
2985   1     0.99689  youtube  
17324  9     0.96677  zisserman
12806  10    0.89518  zurich   

[2313 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 15, 8, 4, 9, 12, 2, 6, 3, 13, 7, 5, 11, 1, 10])